In [30]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from tkinter import *
from tkinter import ttk, messagebox

df = pd.read_csv("content_recommendation_data.csv")
df['success'] = (df['time_spent'] > 300).astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[['scroll_depth', 'time_spent']])

n_clusters = 3  
cluster_model = AgglomerativeClustering(n_clusters=n_clusters)
df['cluster'] = cluster_model.fit_predict(X_scaled)

topic_success = df.groupby(['cluster', 'topic'])['success'].mean().reset_index()

root = Tk()
root.title("Simple Content Recommendation")
root.geometry("450x450")
root.config(bg="#f0f8ff")


frame = Frame(root, padx=20, pady=20, bg="#f0f8ff")
frame.pack(pady=10)


Label(frame, text="Topic:", bg="#f0f8ff").grid(row=0, column=0, sticky=W, pady=5)
topic_var = StringVar()
topic_dropdown = ttk.Combobox(frame, textvariable=topic_var, 
                             values=sorted(df['topic'].unique()), 
                             state="readonly")
topic_dropdown.grid(row=0, column=1)
topic_dropdown.set(df['topic'].unique()[0])


Label(frame, text="Scroll Depth (0-1):", bg="#f0f8ff").grid(row=1, column=0, sticky=W, pady=5)
scroll_entry = Entry(frame, width=10)
scroll_entry.grid(row=1, column=1)
scroll_entry.insert(0, "0.5")

Label(frame, text="Time Spent (seconds):", bg="#f0f8ff").grid(row=2, column=0, sticky=W, pady=5)
time_entry = Entry(frame, width=10)
time_entry.grid(row=2, column=1)
time_entry.insert(0, "200")


result_label = Label(root, text="", font=("Arial", 11), wraplength=400, bg="#f0f8ff")
result_label.pack(pady=10)


recommendation_list = Listbox(root, height=7, width=40)
recommendation_list.pack(pady=10)

def recommend_content():
    try:
        topic = topic_var.get()
        scroll = float(scroll_entry.get())
        time_spent = float(time_entry.get())
        
        if not (0 <= scroll <= 1):
            messagebox.showerror("Error", "Scroll depth must be between 0 and 1")
            return
        

        user_features = scaler.transform([[scroll, time_spent]])
        
        
        cluster_centers = df.groupby('cluster')[['scroll_depth', 'time_spent']].mean()
        cluster_centers_scaled = scaler.transform(cluster_centers)
        
        
        distances = [np.linalg.norm(user_features - center.reshape(1, -1)) 
                    for center in cluster_centers_scaled]
        user_cluster = np.argmin(distances)
        
        
        recommendations = topic_success[topic_success['cluster'] == user_cluster]
        sorted_recommendations = recommendations.sort_values('success', ascending=False)
        
        
        engagement = "High" if time_spent > 300 else "Medium" if time_spent > 100 else "Low"
        result_label.config(
            text=f"Your reading pattern suggests {engagement} engagement.\n"
                 f"Based on similar readers, we recommend these topics:"
        )
        
        
        recommendation_list.delete(0, END)
        for i, (_, row) in enumerate(sorted_recommendations.iterrows(), 1):
            success_pct = row['success'] * 100
            recommendation_list.insert(END, f"{i}. {row['topic']} - {success_pct:.0f}% engagement rate")
            
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers")
    except Exception as e:
        messagebox.showerror("Error", str(e))


recommend_button = ttk.Button(root, text="Get Recommendations", command=recommend_content)
recommend_button.pack(pady=15)


root.mainloop()

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid f